In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from pdb import set_trace
from PIL import Image, ImageDraw, ImageFont
from collections import defaultdict
import xml.etree.ElementTree as ET  
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

from detector.Detector import Detector 

from pdb import set_trace

In [2]:
LP_FROZEN_GRAPH_PATH = '/home/iapp/Desktop/projects/models/workspace/detect_lp/training/faster_rcnn_resnet50_gs/export/frozen_inference_graph.pb'
LP_LABEL_MAP_PATH = '/home/iapp/Desktop/projects/models/workspace/detect_lp/training/faster_rcnn_resnet50_gs/label_map/label_map.pbtxt'
OCR_FROZEN_GRAPH_PATH = '/home/iapp/Desktop/projects/models/workspace/lp_ocr/training/faster_rcnn_resnet50_gs_h150_v2/export/frozen_inference_graph.pb'
OCR_LABEL_MAP_PATH = '/home/iapp/Desktop/projects/models/workspace/lp_ocr/training/faster_rcnn_resnet50_gs_h150_v2/label_map/label_map.pbtxt'

TEST_IMAGES_DIR_PATH =  '/home/iapp/Desktop/projects/lp-ocr-preprocessing/dataturks/pascalVOC/gs-testset/'
#TEST_RESULT_DIR =  '/home/iapp/Desktop/projects/models/workspace/detect_lp/data/trainset/gs/'
TESTSET_RATIO = 1

In [3]:
def sort_detections_left_to_right(single_image_detections):
    return sorted(single_image_detections, key=lambda x: x['box'][0])

In [4]:
def crop_plate(carImage, lpBbox, width=None, height=None):
    xmin, ymin, xmax, ymax = lpBbox
    plateImage = carImage[ymin:ymax, xmin:xmax]
    
    if width == None and height != None:
        width = 'keep-aspect-ratio'
    elif width != None and height == None:
        heigth = 'keep-aspect-ratio'
    seq = iaa.Sequential([iaa.Resize({"width": width, "height": height})])
    return seq(image=plateImage)

In [5]:
def get_actual_chars(image_path, data_dir_path):
    def is_char(object_name):
        return len(object_name) == 1 and ((ord(object_name) in range(ord('0'), ord('9')+1)) or (ord(object_name) in range(ord('ก'), ord('ฮ')+1)))
        
    image_name = os.path.basename(image_path).split('.')[0]
    
    xml_path = os.path.join(data_dir_path, f'{image_name}.xml')
    tree = ET.parse(xml_path)
    object_elem_list = tree.getroot().findall('object')
    
    single_image_ground_truths = []
    for object_elem in object_elem_list:  
        name = object_elem.find('name').text
        if is_char(name):
            ground_truth = {}
            bndbox_elem = object_elem.find('bndbox')

            xmin = int(float(bndbox_elem.find('xmin').text))
            ymin = int(float(bndbox_elem.find('ymin').text))
            xmax = int(float(bndbox_elem.find('xmax').text))
            ymax = int(float(bndbox_elem.find('ymax').text))

            ground_truth['box'] = (xmin, ymin, xmax, ymax)
            ground_truth['label'] = name

            single_image_ground_truths.append(ground_truth) 
    return single_image_ground_truths

In [ ]:
TEST_IMAGE_PATHS =  [f for f in glob.glob(TEST_IMAGES_DIR_PATH + '*.jpg')]

count = 0
with Detector(LP_FROZEN_GRAPH_PATH, LP_LABEL_MAP_PATH) as lp_detector:
    with Detector(OCR_FROZEN_GRAPH_PATH, OCR_LABEL_MAP_PATH) as ocr_detector:
        for image_path in TEST_IMAGE_PATHS: 
            car_image = cv2.imread(image_path)
            car_image = cv2.cvtColor(car_image, cv2.COLOR_BGR2RGB)
            # Detect plate
            plate_detections = lp_detector.detect(car_image, maxDetection=1)
            if len(plate_detections) > 0:
                plate_detection = plate_detections[0]
                plate_image = crop_plate(car_image, plate_detection['box'], height=150)
                
                char_detections = sort_detections_left_to_right(ocr_detector.detect(plate_image, maxDetection=8, minScoreThreshold=.5))
                predict_chars = [det['label'] for det in char_detections]
                char_ground_truths = sort_detections_left_to_right(get_actual_chars(image_path, TEST_IMAGES_DIR_PATH))
                actual_chars = [gt['label'] for gt in char_ground_truths]
                count += (predict_chars == actual_chars)

In [ ]:
count / len(TEST_IMAGE_PATHS)

In [ ]:
char_ground_truths

In [ ]:
char_detections

In [ ]:
image_path